<a href="https://colab.research.google.com/github/YangHyunu/SKT-FLY-AI-5/blob/main/2%EC%A3%BC%EC%B0%A8/23_rnn_binary_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --q ipython-autotime
# %load_ext autotime

time: 308 µs (started: 2024-07-02 05:19:53 +00:00)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras

time: 694 µs (started: 2024-07-02 05:19:53 +00:00)


## RNN
  -영화 리뷰 긍정 부정

In [ ]:
# !unzip /content/movie_review.zip

time: 244 µs (started: 2024-07-02 05:19:53 +00:00)


In [ ]:
train = pd.read_csv('/content/trainData.tsv',delimiter='\t')
test = pd.read_csv('/content/testData.tsv',delimiter='\t')

time: 680 ms (started: 2024-07-02 05:19:53 +00:00)


In [ ]:
train.shape, test.shape

((25000, 3), (25000, 2))

time: 2.19 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
train.iloc[0].review

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

time: 3.51 ms (started: 2024-07-02 05:19:54 +00:00)


## 2.전처리

In [ ]:
# HTML 태그 삭제
from bs4 import BeautifulSoup
bs = BeautifulSoup(train.iloc[0].review,'html.parser')

time: 1.28 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
# bs
bs.text

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

time: 5.12 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
# 숫자, 기호 삭제 <정규 표현 식>
import re
cleaned = re.sub('[^a-zA-Z]',' ',bs.text)
cleaned

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

time: 3.19 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
# 대문자를 소문자로
cleaned = cleaned.lower()
cleaned

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

time: 2.36 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
# 불용어 stopword <큰 의미가 없으면서 문장에서 자주 쓰이는 것 들  uh,mm..
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

time: 9.19 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
from nltk.corpus import stopwords
eng_stop_words = stopwords.words('english')
eng_stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

time: 6.24 ms (started: 2024-07-02 05:38:24 +00:00)


In [ ]:
for word in cleaned.split():
  if word in eng_stop_words:
    print(word)

 # cleaned

with
all
this
down
at
the
with
i
ve
to
his
the
here
and
there
the
and
again
i
just
to
a
into
this
who
i
was
in
the
just
to
up
my
he
is
or
is
which
i
to
at
the
when
it
was
some
of
it
has
about
s
the
and
the
of
are
m
but
of
this
is
all
about
so
you
in
then
you
are
to
this
and
it
some
an
for
to
the
of
this
but
and
most
of
his
that
he
it
for
the
which
if
is
of
him
the
when
it
is
only
on
for
or
so
the
and
is
as
a
all
why
he
so
is
me
because
his
s
that
he
to
it
is
he
who
is
so
i
he
just
s
of
in
this
into
a
and
a
and
the
the
have
had
the
of
a
when
it
to
the
as
with
a
of
them
a
this
is
for
who
on
or
which
i
is
most
if
not
then
it
does
and
off
a
and
s
in
this
is
a
is
of
the
most
to
this
but
is
he
with
all
the
i
ve
this
i
don
t
because
can
be
i
this
for
a
he
is
an
but
or
of
the
most
i
he
is
not
the
time: 18.5 ms (started: 2024-07-02 05:38:42 +00:00)


In [ ]:
def preprocess(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  cleaned = re.sub(r'[^a-zA-Z]', ' ', soup.text)
  cleaned = cleaned.lower()
  cleaned = [word for word in cleaned.split() if word not in eng_stop_words]
  return ' '.join(cleaned)

time: 731 µs (started: 2024-07-02 05:38:46 +00:00)


In [ ]:
train.iloc[0].review

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

time: 22.9 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
preprocess(train.iloc[0].review)

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

time: 41.3 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


time: 269 ms (started: 2024-07-02 05:19:54 +00:00)


In [ ]:
train_clean = train['review'].apply(preprocess)

<ipython-input-69-256122cb1ef0>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence, 'html.parser')


time: 18 s (started: 2024-07-02 05:38:50 +00:00)


In [ ]:
train_clean.head()

0    stuff going moment mj started listening music ...
1    classic war worlds timothy hines entertaining ...
2    film starts manager nicholas bell giving welco...
3    must assumed praised film greatest filmed oper...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

time: 3.98 ms (started: 2024-07-02 05:39:26 +00:00)


## 3. Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_clean)

time: 2.17 s (started: 2024-07-02 05:40:38 +00:00)


In [ ]:
len(tokenizer.word_index)

74065

time: 2.93 ms (started: 2024-07-02 05:39:37 +00:00)


In [ ]:
tokenizer.word_index['world']

74

time: 2.76 ms (started: 2024-07-02 05:39:40 +00:00)


In [ ]:
tokenizer.word_index['hello']

4476

time: 2.63 ms (started: 2024-07-02 05:39:40 +00:00)


## 학습용, 검증용 데이터 분리

In [ ]:
train_lable = train['sentiment']
train_lable

0        1
1        1
2        0
3        0
4        1
        ..
24995    0
24996    0
24997    0
24998    0
24999    1
Name: sentiment, Length: 25000, dtype: int64

time: 5.43 ms (started: 2024-07-02 05:40:55 +00:00)


In [ ]:
from sklearn.model_selection import train_test_split

train_sentence, val_sentence, train_label, val_label = train_test_split(train_clean, train_lable, test_size=0.2, random_state=42)

train_sentence.shape, val_sentence.shape

((20000,), (5000,))

time: 32.8 ms (started: 2024-07-02 05:41:08 +00:00)


In [ ]:
train_sentence.head()

23311    movie plain dumb casting ralph meeker mike ham...
23623    dahmer young confused man dahmer confusing mov...
1020     may saints preserve us movie going help someon...
12645    combination reading novella viewing film inspi...
1533     daniel day lewis left foot gives us one best p...
Name: review, dtype: object

time: 4.83 ms (started: 2024-07-02 05:41:18 +00:00)


In [ ]:
train_sequence = tokenizer.texts_to_sequences(train_sentence)
val_sequence = tokenizer.texts_to_sequences(val_sentence)

time: 1.48 s (started: 2024-07-02 05:41:30 +00:00)


In [ ]:
print(train_sequence[0])

[2, 887, 842, 821, 3003, 14250, 1710, 3772, 24779, 1175, 3, 3297, 1485, 8391, 1710, 3772, 4, 1086, 1052, 168, 26856, 869, 29536, 7405, 160, 1167, 177, 3134, 589, 3772, 1861, 80, 819, 15, 726, 3, 375, 3772, 10390, 3226, 16, 357, 13, 842, 15, 134, 124, 4, 713, 1160, 16758, 3750, 1110, 398, 2, 99, 173, 4304, 178, 155, 72211, 685, 72212, 1673, 16, 118, 2316, 8277, 1200, 53, 2209, 1861, 7592, 34495, 72213, 482, 72214, 1144, 246, 2517, 2191, 2808, 22, 366, 246, 2497, 53, 6810, 205, 138, 839, 87, 17142, 3750, 1110, 16, 118, 6049, 3772, 3911, 4265, 1411, 450, 415, 4265, 172, 1374, 122, 363, 118, 1094, 112, 246, 2265, 1364, 1580, 1797, 16, 101, 32, 170, 106, 30, 614]
time: 951 µs (started: 2024-07-02 05:41:38 +00:00)


## 패딩

In [ ]:
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(train_sequence,
              maxlen=150,
              padding='pre',
              truncating='pre')

val_padded = pad_sequences(val_sequence,
              maxlen=150,
              padding='pre',
              truncating='pre')

train_padded.shape

(20000, 150)

time: 1.31 s (started: 2024-07-02 05:41:58 +00:00)


In [ ]:
print(type(train_padded))
train_label = train_label.to_numpy()
val_label = val_label.to_numpy()
train_label

<class 'numpy.ndarray'>


array([0, 0, 0, ..., 1, 1, 0])

time: 4.59 ms (started: 2024-07-02 05:42:09 +00:00)


## 모델

In [ ]:
EMBEDDING_DIM = 300
VAOCA_SIZE = len(tokenizer.word_index)+1
VAOCA_SIZE

74067

time: 5.65 ms (started: 2024-07-02 05:42:16 +00:00)


In [ ]:
from keras import layers

model = keras.Sequential([
    layers.Embedding(VAOCA_SIZE, EMBEDDING_DIM, input_length=150),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(128),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

time: 1.89 s (started: 2024-07-02 05:42:24 +00:00)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          22220100  
                                                                 
 lstm (LSTM)                 (None, 150, 128)          219648    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 22575493 (86.12 MB)
Trainable params: 2257

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

time: 18.5 ms (started: 2024-07-02 05:42:45 +00:00)


In [ ]:
EPOCHS = 1
BATCH_SIZE = 32

history = model.fit(
    train_padded, train_label,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    # validation_split = 0.2
    validation_data=(val_padded, val_label)
)

625/625 [==============================] - 83s 121ms/step - loss: 0.3752 - accuracy: 0.8346 - val_loss: 0.3274 - val_accuracy: 0.8706
time: 1min 23s (started: 2024-07-02 05:42:52 +00:00)
